# Section 3: Create Circuits - Practice Questions

**Exam Weight**: 18% (~12 questions) | **Difficulty**: High | **Must Master**: ✅✅✅✅

⚠️ **HIGHEST WEIGHTED SECTION** - Focus heavily here!

---

## 🎯 Key Traps to Watch For:

| Trap | Wrong Assumption | Correct Understanding |
|------|------------------|----------------------|
| c_if() | Still works | DEPRECATED! Use `if_test()` context manager |
| compose() | Returns new circuit | `inplace=False` default - must assign result! |
| Parameter binding | Modifies circuit | Returns NEW circuit - `bound_qc = qc.assign_parameters({p: 0.5})` |
| to_gate() | Preserves name | Creates anonymous gate - use `qc.to_gate(label='name')` |
| append() | Adds gate object | `qc.append(gate, [qubits])` - needs qubit list! |
| ParameterVector | Like list | `params = ParameterVector('θ', 3)` - use params[0], params[1] |

> 📖 See section_3_create_circuits/README.md for full concepts

---

## 📚 Topics Covered (from Section Notebooks):

### Circuit Basics (`circuit_basics.ipynb`)
- **QuantumCircuit**: `QuantumCircuit(n)`, `QuantumCircuit(n, m)`
- **Named Registers**: `QuantumRegister()`, `ClassicalRegister()`
- **Circuit Properties**: `.num_qubits`, `.num_clbits`, `.depth()`, `.size()`
- **Measurements**: `measure()`, `measure_all()`

### Circuit Composition (`circuit_composition.ipynb`)
- **compose()**: `qc1.compose(qc2)` - sequential composition
- **tensor()**: `qc1.tensor(qc2)` - parallel composition
- **append()**: `qc.append(gate, qubits, clbits)`
- **to_gate()**: Convert circuit to reusable gate
- **to_instruction()**: Convert to instruction

### Parameterized Circuits (`parameterized_circuits.ipynb`)
- **Parameter**: `Parameter('θ')` - single parameter
- **ParameterVector**: `ParameterVector('θ', n)` - array of parameters
- **assign_parameters()**: `qc.assign_parameters({p: value})`
- **Partial binding**: Bind subset of parameters

### Circuit Library (`circuit_library.ipynb`)
- **Standard gates**: `QFT`, `IQFT`
- **Ansätze**: `TwoLocal`, `EfficientSU2`, `RealAmplitudes`
- **Decomposition**: `.decompose()` method

### Classical Control (`classical_control.ipynb`)
- **c_if()** (DEPRECATED): Legacy conditional gates
- **Modern control flow**: Use dynamic circuits instead

### Dynamic Circuits (`dynamic_circuits.ipynb`)
- **if_test()**: `with qc.if_test((clbit, value)):`
- **for_loop()**: `with qc.for_loop(range(n)):`
- **while_loop()**: `with qc.while_loop((clbit, value)):`
- **switch()**: `with qc.switch(clbits):`

In [ ]:
# Setup - Run this first!
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import RYGate, RealAmplitudes, EfficientSU2
from qiskit.quantum_info import Statevector
import numpy as np
%matplotlib inline
print("✅ Setup complete!")

---
## Part 1: Circuit Basics & Registers

| Constructor | Description |
|-------------|-------------|
| `QuantumCircuit(n)` | n qubits, no classical |
| `QuantumCircuit(n, m)` | n qubits, m classical bits |
| `QuantumCircuit(qr, cr)` | From named registers |

### Q1: Create circuit with named registers

In [ ]:
# Your solution: Create circuit with QuantumRegister('data', 2) and ClassicalRegister('result', 2)

In [ ]:
# Solution Q1
qr = QuantumRegister(2, 'data')
cr = ClassicalRegister(2, 'result')
qc = QuantumCircuit(qr, cr)

qc.h(qr[0])
qc.cx(qr[0], qr[1])
qc.measure(qr, cr)

print(qc.draw())
print(f"\nQuantum registers: {qc.qregs}")
print(f"Classical registers: {qc.cregs}")

### Q2: Add registers to existing circuit

In [ ]:
# Your solution: Create empty circuit, add registers with add_register()

In [ ]:
# Solution Q2
qc = QuantumCircuit()

# Add registers dynamically
qr1 = QuantumRegister(2, 'main')
qr2 = QuantumRegister(2, 'ancilla')
cr = ClassicalRegister(2, 'output')

qc.add_register(qr1)
qc.add_register(qr2)
qc.add_register(cr)

qc.h(qr1[0])
qc.cx(qr1[0], qr2[0])

print(qc.draw())
print(f"Total qubits: {qc.num_qubits}")

---
## Part 2: Circuit Composition

| Method | Effect |
|--------|--------|
| `compose(other, qubits)` | Append other circuit |
| `compose(other, front=True)` | Prepend other circuit |
| `tensor(other)` | Parallel circuits (⊗) |
| `append(gate, qubits)` | Add single gate |

### Q3: compose() with qubit mapping

In [ ]:
# Your solution: Compose a 2-qubit circuit onto specific qubits of a 3-qubit circuit

In [ ]:
# Solution Q3
# Main 3-qubit circuit
main = QuantumCircuit(3)
main.h(0)

# Sub-circuit to compose
sub = QuantumCircuit(2)
sub.cx(0, 1)
sub.t(1)

# Compose onto qubits [1, 2]
result = main.compose(sub, qubits=[1, 2])

print("Main:")
print(main.draw())
print("\nSub:")
print(sub.draw())
print("\nComposed (sub on qubits 1,2):")
print(result.draw())

### Q4: compose() with front=True

In [ ]:
# Your solution: Show difference between front=True and front=False

In [ ]:
# Solution Q4
qc1 = QuantumCircuit(1)
qc1.h(0)

qc2 = QuantumCircuit(1)
qc2.x(0)

# front=False (default): qc2 goes AFTER qc1
back = qc1.compose(qc2, front=False)
print("front=False (X after H):")
print(back.draw())

# front=True: qc2 goes BEFORE qc1
front = qc1.compose(qc2, front=True)
print("\nfront=True (X before H):")
print(front.draw())

### Q5: tensor() for parallel circuits

In [ ]:
# Your solution: Create parallel combination of two circuits using tensor()

In [ ]:
# Solution Q5
qc1 = QuantumCircuit(1)
qc1.h(0)

qc2 = QuantumCircuit(2)
qc2.cx(0, 1)

# tensor = parallel combination
combined = qc1.tensor(qc2)

print("Circuit 1:")
print(qc1.draw())
print("\nCircuit 2:")
print(qc2.draw())
print("\nTensor product (parallel):")
print(combined.draw())
print(f"Total qubits: {combined.num_qubits}")

---
## Part 3: Parameterized Circuits

| Class | Usage |
|-------|-------|
| `Parameter('θ')` | Single parameter |
| `ParameterVector('θ', n)` | Array of n parameters |

| Method | Description |
|--------|-------------|
| `assign_parameters({p: val})` | Bind with dict |
| `assign_parameters([vals])` | Bind with list |

### Q6: Create parameterized circuit

In [ ]:
# Your solution: Create circuit with Parameter θ for RY rotation

In [ ]:
# Solution Q6
theta = Parameter('θ')

qc = QuantumCircuit(1)
qc.ry(theta, 0)

print("Parameterized circuit:")
print(qc.draw())
print(f"\nParameters: {qc.parameters}")

### Q7: Bind parameters with dict

In [ ]:
# Your solution: Bind θ = π/2 using assign_parameters with dict

In [ ]:
# Solution Q7
theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.ry(theta, 0)

# Bind using dictionary
bound_qc = qc.assign_parameters({theta: np.pi / 2})

print("Bound circuit:")
print(bound_qc.draw())

sv = Statevector(bound_qc)
print(f"\nState: {sv}")

### Q8: ParameterVector for multiple parameters

In [ ]:
# Your solution: Create VQE-style ansatz with ParameterVector

In [ ]:
# Solution Q8
params = ParameterVector('θ', length=4)

qc = QuantumCircuit(2)
qc.ry(params[0], 0)
qc.ry(params[1], 1)
qc.cx(0, 1)
qc.ry(params[2], 0)
qc.ry(params[3], 1)

print("VQE-style ansatz:")
print(qc.draw())
print(f"\nParameters: {list(qc.parameters)}")

# Bind all at once with list
bound = qc.assign_parameters([0.1, 0.02, 0.3, 0.4])
print("\nBound circuit:")
print(bound.draw())

### Q8b: ⚠️ EXAM TRAP - assign_parameters() Returns NEW Circuit!

```python
# ❌ WRONG - Original circuit unchanged!
qc.assign_parameters({theta: 0.5})
print(qc.parameters)  # Still has theta!

# ✅ CORRECT - Capture the returned circuit
bound_qc = qc.assign_parameters({theta: 0.5})
print(bound_qc.parameters)  # Empty set - bound!
```

In [ ]:
# Solution Q8b: Demonstrating the Trap
theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.ry(theta, 0)

# WRONG WAY - doesn't capture result
qc.assign_parameters({theta: np.pi/2})  # Returns new circuit but ignored!
print(f"Original still has parameter: {qc.parameters}")

# CORRECT WAY
bound_qc = qc.assign_parameters({theta: np.pi/2})
print(f"Bound circuit has no parameters: {bound_qc.parameters}")

### Q8c: Standard VQE Ansatz Circuits from Library (EXAM TESTED!)

In [ ]:
# Solution Q8c: Library Ansatz Circuits
# RealAmplitudes - Common VQE ansatz
ansatz1 = RealAmplitudes(num_qubits=2, reps=1)
print("RealAmplitudes ansatz:")
print(ansatz1.draw())
print(f"Parameters: {ansatz1.num_parameters}")

# EfficientSU2 - More expressive ansatz
ansatz2 = EfficientSU2(num_qubits=2, reps=1)
print("\nEfficientSU2 ansatz:")
print(ansatz2.draw())
print(f"Parameters: {ansatz2.num_parameters}")

# Bind and verify
bound = ansatz1.assign_parameters(np.random.random(ansatz1.num_parameters))
print("\nBound RealAmplitudes:")
print(bound.draw())

---
## Part 4: Circuit Methods

| Method | Returns |
|--------|--------|
| `to_gate()` | Gate object |
| `to_instruction()` | Instruction object |
| `inverse()` | Reversed circuit |
| `decompose()` | Break into basis gates |
| `copy_empty_like()` | Same structure, no gates |

### Q9: Convert circuit to gate

In [ ]:
# Your solution: Create Bell state prep, convert to gate, use in larger circuit

In [ ]:
# Solution Q9
# Create sub-circuit
bell_prep = QuantumCircuit(2, name='Bell')
bell_prep.h(0)
bell_prep.cx(0, 1)

# Convert to gate
bell_gate = bell_prep.to_gate()

# Use in larger circuit
main = QuantumCircuit(4)
main.append(bell_gate, [0, 1])  # Apply to qubits 0,1
main.append(bell_gate, [2, 3])  # Apply to qubits 2,3

print("Main circuit with Bell gates:")
print(main.draw())
print("\nDecomposed:")
print(main.decompose().draw())

### Q10: Controlled gate creation

In [ ]:
# Your solution: Create controlled-RY gate using .control()

In [ ]:
# Solution Q10
from qiskit.circuit.library import RYGate

# Create RY gate and make it controlled
ry_gate = RYGate(np.pi/4)
controlled_ry = ry_gate.control(1)  # 1 control qubit

qc = QuantumCircuit(2)
qc.h(0)  # Control qubit in superposition
qc.append(controlled_ry, [0, 1])  # [control, target]

print(qc.draw())
print("\nDecomposed:")
print(qc.decompose().draw())

---
## Part 5: Classical Control

⚠️ **EXAM CRITICAL**: `c_if()` is DEPRECATED in favor of `if_test()`!

| Old (Deprecated) | New (Section 9) |
|------------------|----------------|
| `gate.c_if(creg, val)` | `with qc.if_test((clbit, val)):` |

### Q11: Classical conditional with c_if (legacy)

In [ ]:
# Your solution: Apply X gate conditionally if measurement result is 1

In [ ]:
# Solution Q11 (Legacy c_if - still on some exams)
qc = QuantumCircuit(2, 1)
qc.h(0)
qc.measure(0, 0)

# Apply X to qubit 1 if classical bit 0 equals 1
# In Qiskit 1.0+, use with statement for if_test instead of c_if
with qc.if_test((qc.clbits[0], 1)):
	qc.x(1)

print(qc.draw())
print("\n⚠️ Note: c_if is deprecated, use if_test() (Section 9)")

### Q12: Multiple classical conditions

In [ ]:
# Your solution: Condition on classical register value

In [ ]:
# Solution Q12
qc = QuantumCircuit(3, 2)
qc.h([0, 1])
qc.measure([0, 1], [0, 1])

# Apply Z to qubit 2 if classical register equals 3 (binary '11')
with qc.if_test((qc.cregs[0], 3)):
    qc.z(2)

print(qc.draw())
print("\nZ applied only when both measurements are 1")

---
## Part 6: Circuit Copy and Metadata

### Q13: copy_empty_like()

In [ ]:
# Your solution: Create empty copy of circuit preserving structure

In [ ]:
# Solution Q13
qr = QuantumRegister(2, 'q')
cr = ClassicalRegister(2, 'c')
original = QuantumCircuit(qr, cr)
original.h(0)
original.cx(0, 1)
original.measure_all()

# Create empty copy (same registers, no gates)
empty_copy = original.copy_empty_like()

print("Original:")
print(original.draw())
print("\nEmpty copy (same structure):")
print(empty_copy.draw())
print(f"Same registers: {original.qregs == empty_copy.qregs}")

### Q14: Circuit metadata

In [ ]:
# Your solution: Add and access circuit metadata

In [ ]:
# Solution Q14
qc = QuantumCircuit(2, name='my_circuit')
qc.h(0)
qc.cx(0, 1)

# Add metadata
qc.metadata = {'purpose': 'Bell state', 'version': '1.0'}

print(f"Circuit name: {qc.name}")
print(f"Num qubits: {qc.num_qubits}")
print(f"Num clbits: {qc.num_clbits}")
print(f"Metadata: {qc.metadata}")
print(f"Global phase: {qc.global_phase}")

---
## ✅ Section 3 Checklist

**Circuit Creation**:
- [ ] `QuantumCircuit(n, m)`, `QuantumRegister`, `ClassicalRegister`
- [ ] `add_register()`

**Composition**:
- [ ] `compose(other, qubits=[...])` - sequential
- [ ] `compose(other, front=True)` - prepend
- [ ] `tensor(other)` - parallel
- [ ] `append(gate, qubits)`

**Parameters**:
- [ ] `Parameter('θ')`, `ParameterVector('θ', n)`
- [ ] `assign_parameters({p: val})` or `assign_parameters([vals])`

**Circuit Methods**:
- [ ] `to_gate()`, `to_instruction()`
- [ ] `gate.control(n)` - create controlled version
- [ ] `inverse()`, `decompose()`
- [ ] `copy_empty_like()`

**Classical Control**:
- [ ] `c_if(creg, value)` (deprecated but tested)
- [ ] Modern: `if_test()` (Section 9)